In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [7]:
import numpy as np

from initialize import initialize
from genregparam import linparams

In [9]:
capacity = 50
intensity = 1.5
slopes = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [10]:
a, b, c, d = initialize(capacity, intensity, rates_init)
e, f = linparams(capacity, intensity, slopes, rates_init)

In [12]:
f

array([[[22.62227621, 32.40628061, 37.809649  ],
        [26.12394516, 35.67537805, 42.77259624],
        [28.92817251, 34.40896481, 37.17615239],
        [25.62695653, 34.34543388, 33.91875214],
        [20.42850943, 23.61110127, 30.06144173],
        [23.57635566, 26.70233184, 33.22054347],
        [20.47008763, 26.39493584, 23.49121457]],

       [[28.51441076, 43.84826051, 39.34586602],
        [24.44204377, 34.88606404, 51.58742009],
        [28.14581111, 30.49564832, 33.60918932],
        [37.12851546, 32.89550617, 44.81204469],
        [25.92395201, 30.65840075, 34.42028485],
        [25.37732779, 33.06529243, 40.60196774],
        [26.72282129, 34.2489876 , 37.61645861]]])